In [11]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

import numpy as np

In [20]:
def pascal(nrows):
    a = [1]
    yield(a)
    
    for i in range(nrows):
        a = np.convolve(a, [1,1])
        yield(a)

In [21]:
for row in pascal(5):
    print(row)

[1]
[1 1]
[1 2 1]
[1 3 3 1]
[1 4 6 4 1]
[ 1  5 10 10  5  1]


In [22]:
row

array([ 1,  5, 10, 10,  5,  1])

In [23]:
ind = range(len(row))

range(0, 6)

In [24]:
from itertools import combinations

In [71]:
def all_combos(s, n):
    for r in range(1, n):
        for comb in combinations(s, r):
            yield comb

In [72]:
n = 3
ind = set(range(n))

{0, 1, 2}

In [73]:
for comb in all_combos(ind, n):
    print(comb)

(0,)
(1,)
(2,)
(0, 1)
(0, 2)
(1, 2)


In [74]:
for comb in all_combos(ind, n+1):
    print(comb)

(0,)
(1,)
(2,)
(0, 1)
(0, 2)
(1, 2)
(0, 1, 2)


In [77]:
def all_partitions(s):
    for numer in all_combos(s, len(s)):
        rest = ind - set(numer)
        for denom in all_combos(rest, len(rest)+1):
            yield numer, denom

In [78]:
for part in all_partitions(ind):
    print(part)

((0,), (1,))
((0,), (2,))
((0,), (1, 2))
((1,), (0,))
((1,), (2,))
((1,), (0, 2))
((2,), (0,))
((2,), (1,))
((2,), (0, 1))
((0, 1), (2,))
((0, 2), (1,))
((1, 2), (0,))
